In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"varun19424","key":"098cf3e8782631feb8978e9981916760"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list

ref                                                       title                                             size  lastUpdated                 downloadCount  voteCount  usabilityRating  
--------------------------------------------------------  ------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
urvishahir/electric-vehicle-specifications-dataset-2025   Electric Vehicle Specs Dataset (2025)            16631  2025-06-20 07:14:56.777000           4903        119  1.0              
imtkaggleteam/health-insurance-dataset                    Health Insurance Dataset                         20398  2025-07-05 11:59:00.730000            760         23  1.0              
divyaraj2006/social-media-engagement                      social media engagement                           2142  2025-07-02 15:34:21.323000           2019         42  1.0              
darrendube/airlinedataset                                 Airline Perf

In [ ]:
!kaggle datasets download -d rijulshr/pneumoniamnist

Dataset URL: https://www.kaggle.com/datasets/rijulshr/pneumoniamnist
License(s): MIT
  0% 0.00/3.39M [00:00<?, ?B/s]
100% 3.39M/3.39M [00:00<00:00, 635MB/s]


In [ ]:
!unzip pneumoniamnist.zip -d data/

Archive:  pneumoniamnist.zip
  inflating: data/pneumoniamnist.npz  


## Load the Dataset from .npz

In [ ]:
import numpy as np

# Load dataset
data = np.load("data/pneumoniamnist.npz")

# Extract splits
x_train, y_train = data["train_images"], data["train_labels"]
x_val, y_val = data["val_images"], data["val_labels"]
x_test, y_test = data["test_images"], data["test_labels"]

print("Train:", x_train.shape, y_train.shape)
print("Val:", x_val.shape, y_val.shape)
print("Test:", x_test.shape, y_test.shape)

Train: (3882, 28, 28) (3882, 1)
Val: (524, 28, 28) (524, 1)
Test: (624, 28, 28) (624, 1)


## Preprocess for InceptionV3 (Reshape, Resize, RGB)

In [ ]:

import tensorflow as tf

def preprocess(images):
    # Expand grayscale to RGB, resize to 299x299
    images = np.expand_dims(images, axis=-1)  # (B, 28, 28, 1)
    images = tf.image.grayscale_to_rgb(tf.convert_to_tensor(images))
    images = tf.image.resize(images, [299, 299])
    return images / 255.0

x_train = preprocess(x_train)
x_val = preprocess(x_val)
x_test = preprocess(x_test)

# Labels to float32 for binary classification
y_train = y_train.astype('float32')
y_val = y_val.astype('float32')
y_test = y_test.astype('float32')

## Build InceptionV3 Model (Transfer Learning)

In [ ]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=(299, 299, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
output = Dense(1, activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(1e-4), loss="binary_crossentropy", metrics=["accuracy"])

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


## Train the Model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Create the callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=3)

checkpoint_cb = ModelCheckpoint(
    "best_model.h5",                # File path to save the best model
    save_best_only=True,            # Only save the best version
    monitor="val_loss",             # Monitor validation loss
    mode="min",                     # Minimize val_loss
    verbose=1
)

# Now this will work
model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=25,
    batch_size=32,
    callbacks=[early_stop, checkpoint_cb]
)

Epoch 1/25
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step - accuracy: 0.6637 - loss: 0.6419
Epoch 1: val_loss improved from inf to 0.75410, saving model to best_model.h5


122/122 ━━━━━━━━━━━━━━━━━━━━ 71s 385ms/step - accuracy: 0.6648 - loss: 0.6408 - val_accuracy: 0.7424 - val_loss: 0.7541
Epoch 2/25
121/122 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 0.9058 - loss: 0.3429
Epoch 2: val_loss improved from 0.75410 to 0.57606, saving model to best_model.h5


122/122 ━━━━━━━━━━━━━━━━━━━━ 35s 137ms/step - accuracy: 0.9057 - loss: 0.3429 - val_accuracy: 0.7424 - val_loss: 0.5761
Epoch 3/25
121/122 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step - accuracy: 0.8996 - loss: 0.2896
Epoch 3: val_loss improved from 0.57606 to 0.48273, saving model to best_model.h5


122/122 ━━━━━━━━━━━━━━━━━━━━ 21s 144ms/step - accuracy: 0.8997 - loss: 0.2894 - val_accuracy: 0.7424 - val_loss: 0.4827
Epoch 4/25
121/122 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 0.9043 - loss: 0.2582
Epoch 4: val_loss improved from 0.48273 to 0.41059, saving model to best_model.h5


122/122 ━━━━━━━━━━━━━━━━━━━━ 20s 137ms/step - accuracy: 0.9043 - loss: 0.2580 - val_accuracy: 0.7710 - val_loss: 0.4106
Epoch 5/25
121/122 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - accuracy: 0.9155 - loss: 0.2225
Epoch 5: val_loss improved from 0.41059 to 0.37086, saving model to best_model.h5


...

122/122 ━━━━━━━━━━━━━━━━━━━━ 21s 135ms/step - accuracy: 0.9477 - loss: 0.1353 - val_accuracy: 0.8740 - val_loss: 0.2510
Epoch 19/25
121/122 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 0.9416 - loss: 0.1467
Epoch 19: val_loss improved from 0.25101 to 0.24786, saving model to best_model.h5


122/122 ━━━━━━━━━━━━━━━━━━━━ 20s 135ms/step - accuracy: 0.9416 - loss: 0.1467 - val_accuracy: 0.8760 - val_loss: 0.2479
Epoch 20/25
121/122 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 0.9499 - loss: 0.1327
Epoch 20: val_loss did not improve from 0.24786
122/122 ━━━━━━━━━━━━━━━━━━━━ 16s 131ms/step - accuracy: 0.9499 - loss: 0.1328 - val_accuracy: 0.8740 - val_loss: 0.2513
Epoch 21/25
121/122 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 0.9428 - loss: 0.1382
Epoch 21: val_loss did not improve from 0.24786
122/122 ━━━━━━━━━━━━━━━━━━━━ 16s 131ms/step - accuracy: 0.9428 - loss: 0.1382 - val_accuracy: 0.8740 - val_loss: 0.2582
Epoch 22/25
121/122 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 0.9507 - loss: 0.1272
Epoch 22: val_loss improved from 0.24786 to 0.23909, saving model to best_model.h5


122/122 ━━━━━━━━━━━━━━━━━━━━ 21s 136ms/step - accuracy: 0.9506 - loss: 0.1273 - val_accuracy: 0.8779 - val_loss: 0.2391
Epoch 23/25
121/122 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.9478 - loss: 0.1397
Epoch 23: val_loss did not improve from 0.23909
122/122 ━━━━━━━━━━━━━━━━━━━━ 17s 138ms/step - accuracy: 0.9478 - loss: 0.1396 - val_accuracy: 0.8702 - val_loss: 0.2459
Epoch 24/25
121/122 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.9477 - loss: 0.1416
Epoch 24: val_loss improved from 0.23909 to 0.23449, saving model to best_model.h5


122/122 ━━━━━━━━━━━━━━━━━━━━ 19s 160ms/step - accuracy: 0.9477 - loss: 0.1416 - val_accuracy: 0.8817 - val_loss: 0.2345
Epoch 25/25
121/122 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 0.9430 - loss: 0.1345
Epoch 25: val_loss improved from 0.23449 to 0.23039, saving model to best_model.h5


122/122 ━━━━━━━━━━━━━━━━━━━━ 17s 143ms/step - accuracy: 0.9431 - loss: 0.1345 - val_accuracy: 0.8836 - val_loss: 0.2304


## Evaluate and Print Classification Metrics

In [ ]:
from sklearn.metrics import classification_report

y_pred = (model.predict(x_test) > 0.5).astype("int32")

print(classification_report(y_test, y_pred, target_names=["Normal", "Pneumonia"]))

20/20 ━━━━━━━━━━━━━━━━━━━━ 26s 839ms/step
              precision    recall  f1-score   support

      Normal       0.97      0.40      0.57       234
   Pneumonia       0.73      0.99      0.84       390

    accuracy                           0.77       624
   macro avg       0.85      0.70      0.71       624
weighted avg       0.82      0.77      0.74       624



In [ ]:
model.save("inception_pneumonia_model.h5")

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_cb = ModelCheckpoint("best_model.h5", save_best_only=True)

In [ ]:
model.load_weights("best_model.h5")